In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-darwin-amd64.zip
!unzip ngrok-v3-stable-darwin-amd64.zip

In [ ]:
!./ngrok config add-authtoken 1tHaGIL1YdMNniMQKsAzqSgsiUb_7AgZfgwYUTdCzFGva4KE8

In [1]:
from Game2048Env import Game2048Env
from MCTS2048 import MCTS2048
from Policy2048 import create_2048_policy_model
from tqdm.notebook import tqdm, trange
import numpy as np
import threading
from pyngrok import ngrok
from webapp2048.WebApp import socketio, app, set_public_url, call_add_step

2023-10-22 18:29:10.490459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Function to run the Flask app with SocketIO
def run():
    socketio.run(app, host='0.0.0.0', port=5001, debug=True, use_reloader=False)

def run_async():
    # Start the Flask app in a separate thread
    threading.Thread(target=run).start()

    # Set up ngrok tunnel
    public_url = ngrok.connect(5001).public_url
    set_public_url(public_url)
    print('Public URL:', public_url)


In [ ]:
def train_2048(model, env, mcts, epochs, games_per_epoch, mcts_iterations, batch_size):
    """
    Train the model for the 2048 game using the given environment and MCTS.

    Parameters:
    - model: Neural network model
    - env: 2048 game environment
    - mcts: MCTS object
    - epochs: Number of training epochs
    - games_per_epoch: Number of games to play per epoch
    - mcts_iterations: Number of MCTS iterations per move
    - batch_size: Batch size for training the model

    Returns:
    - model: Trained model
    """

    for epoch in trange(epochs):
        # Lists to store training data
        states, action_masks, action_probs, values = [], [], [], []

        for game in range(games_per_epoch):
            state = mcts.reset_env()
            action_mask = env.legal_actions_mask(state)
            game_value = 0
            step = 0

            while True:
                step += 1
                action = mcts.search(mcts.root)

                if not env.is_action_legal(env.board, action):
                    print("illegal")

                next_state, reward, done, extra_info = env.step(action)
                next_random_action = extra_info['random_action']
                next_action_mask = env.legal_actions_mask(next_state)
                mcts.move_to_child(next_state, action, next_random_action)

                state_action_probs = np.zeros(4)
                state_action_probs[action] = 1

                states.append(state)
                values.append(reward)
                action_probs.append(state_action_probs)
                action_masks.append(action_mask)

                state = next_state
                action_mask = next_action_mask


                call_add_step({
                    'epoch_number': epoch,
                    'game_number': game,
                    'action': action,
                    'next_board': next_state.tolist()
                })

                env.render()

                if done:
                    break
            # Assign final game rewards
            winner_value = 1 if game_value > 0 else -1
            values = [winner_value] * len(values)

        # Convert training data into arrays
        states = np.array(states).reshape(-1, 4, 4, 1)
        action_probs = np.array(action_probs)
        values = np.array(values).reshape(-1, 1)
        action_masks = np.array(action_masks).reshape(-1, 4)

        # Train the model
        model.fit([states, action_masks], [values, action_probs], batch_size=batch_size, epochs=1)

    return model

run_async()

# Hyperparameters
epochs = 1000
games_per_epoch = 10
mcts_iterations = 10
batch_size = 32

# Create the 2048 environment, model, and MCTS
env = Game2048Env(enable_pygame=True)
model = create_2048_policy_model()
mcts = MCTS2048(model, mcts_iterations, env)

# Train the model
trained_model = train_2048(model, env, mcts, epochs, games_per_epoch, mcts_iterations, batch_size)

(41821) wsgi starting up on http://0.0.0.0:5001
t=2023-10-22T18:29:16-0500 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path="/Users/michaelking/Library/Application Support/ngrok/ngrok.yml" legacy_path=/Users/michaelking/.ngrok2/ngrok.yml


Public URL: https://6d77-173-20-233-125.ngrok-free.app


  0%|          | 0/1000 [00:00<?, ?it/s]

(41821) accepted ('127.0.0.1', 51894)
127.0.0.1 - - [22/Oct/2023 18:29:17] "GET /socket.io/?EIO=4&transport=polling&t=OjPkAs0 HTTP/1.1" 200 330 0.000638
127.0.0.1 - - [22/Oct/2023 18:29:17] "POST /socket.io/?EIO=4&transport=polling&t=OjPkAs5&sid=bkxGh7WyumKaHXGnAAAA HTTP/1.1" 200 219 0.001645
127.0.0.1 - - [22/Oct/2023 18:29:17] "GET /socket.io/?EIO=4&transport=polling&t=OjPkAsD&sid=bkxGh7WyumKaHXGnAAAA HTTP/1.1" 200 265 0.000144
(41821) accepted ('127.0.0.1', 51918)
127.0.0.1 - - [22/Oct/2023 18:29:17] "GET /socket.io/?EIO=4&transport=polling&t=OjPkAsG&sid=bkxGh7WyumKaHXGnAAAA HTTP/1.1" 200 233 0.000122
(41821) accepted ('127.0.0.1', 53277)
173.20.233.125,127.0.0.1 - - [22/Oct/2023 18:29:17] "POST /add_step HTTP/1.1" 200 166 0.001135


<Response [200]>


...

<Response [200]>


(41821) accepted ('127.0.0.1', 65482)
173.20.233.125,127.0.0.1 - - [22/Oct/2023 18:55:14] "POST /add_step HTTP/1.1" 200 166 0.000768


<Response [200]>


(41821) accepted ('127.0.0.1', 50253)
173.20.233.125,127.0.0.1 - - [22/Oct/2023 18:55:15] "POST /add_step HTTP/1.1" 200 166 0.000649


<Response [200]>
